# Deribit Api Management

## Api configuration

In [137]:
import pprint
import nest_asyncio
import asyncio
import websockets
import json
import pandas as pd
import datetime as dt
import time

ACTUAL_TIME = round(time.time() * 1000) 
# End time is actual time

nest_asyncio.apply()

async def call_api(msg):
   async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           return response

def async_loop(api, message):
    return asyncio.get_event_loop().run_until_complete(api(message))


## Simple api calls (getting information)

In [138]:
def retrieve_historic_data(start, end, instrument, timeframe = '1'):
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 833,
            "method": "public/get_tradingview_chart_data",
            "params": {
                "instrument_name": instrument,
                "start_timestamp": start,
                "end_timestamp": end,
                "resolution": timeframe
            }
        }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def getSummary (currency = 'BTC', type = 'option'):
    msg = \
    {
        "jsonrpc" : "2.0",
        "id" : 9344,
        "method" : "public/get_book_summary_by_currency",
        "params" : {
            "currency" : currency,
            "kind" : type
        }
    }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def getInstruments (currency = "BTC", kind = 'option' , expired = False):
    msg = \
    {
        "jsonrpc" : "2.0",
        "id" : 7617,
        "method" : "public/get_instruments",
        "params" : {
            "currency" : "BTC",
            "kind" : "future",
            "expired" : False
        }
    }

    resp = async_loop(call_api, json.dumps(msg))

    return resp    

    
def epochToDate(epoch):
    return time.strftime('%d-%m-%Y %H:%M',time.localtime(epoch/1000))





## Calling Api

In [139]:
instruments_json_resp = getSummary()

## Cleaning data

In [148]:
def jsonToDataFrame(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    return df
    
def historyCleaning(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    df['ticks'] = df.ticks / 1000
    # df['timestamp'] = [dt.datetime.fromtimestamp(date) for date in df.ticks]

    return df

def dateConverter(date):
    dict = {
        'JAN' : 1 ,
        'FEB' : 2 ,
        'MAR' : 3 ,
        'APR' : 4 ,
        'MAY' : 5 ,
        'JUN' : 6 ,
        'JUL' : 7 ,
        'AUG' : 8 ,
        'SEP' : 9 ,
        'OCT' : 10 ,
        'NOV' : 11 ,
        'DEC' : 12 ,
            
    }
    

    return '{}/{}/{}'.format(date[0:-5], dict[date[-5:-2]], date[-2:])

def instrumentsCleaning(json_resp):
    df = jsonToDataFrame(json_resp)
    

    df = df.sort_values(["instrument_name"]) 
    df['expiry_date'] = [j.split('-')[1] for j in df['instrument_name']]
    # df['expiration_timestamp'] = df['expiration_timestamp'].apply(epochToDate)
    df['expiry_date'] = [dateConverter(j) for j in df['expiry_date']]
    df['expiry_month'] = [j.split("/")[1]  for j in df['expiry_date']]
    # df.drop(['base_currency','creation_timestamp','kind','option_type','quote_currency','min_trade_amount','is_active','block_trade_commission'],axis = 1 , inplace=True)
    df = df[['instrument_name','mark_price','volume','estimated_delivery_price','expiry_date','expiry_month']]

    

    return df



In [149]:
instrument_df = instrumentsCleaning(instruments_json_resp)
instrument_df



,instrument_name,mark_price,volume,estimated_delivery_price,expiry_date,expiry_month
36,BTC-15FEB22-39000-C,0.075672,5.5,42161.61,15/2/22,2
89,BTC-15FEB22-39000-P,0.000726,30.5,42161.61,15/2/22,2
108,BTC-15FEB22-40000-C,0.051954,4.0,42161.61,15/2/22,2
220,BTC-15FEB22-40000-P,0.000728,459.4,42161.61,15/2/22,2
167,BTC-15FEB22-41000-C,0.029257,177.1,42161.61,15/2/22,2
...,...,...,...,...,...,...
132,BTC-4MAR22-50000-P,0.253427,2.0,42161.61,4/3/22,3
68,BTC-4MAR22-55000-C,0.043397,0.0,42161.61,4/3/22,3
143,BTC-4MAR22-55000-P,0.347942,0.0,42161.61,4/3/22,3
84,BTC-4MAR22-60000-C,0.027544,0.0,42161.61,4/3/22,3


## Separating Calls and Puts

In [153]:
instrument_df['optionType'] = [j[-1] for j in instrument_df['instrument_name']]
# instrument_df['expiry_date'] = [j[5:-8] for j in instrument_df['expiry_date']]

put_df = instrument_df[instrument_df['optionType'] == 'P'].drop('optionType',axis=1)
call_df = instrument_df[instrument_df['optionType'] == 'C'].drop('optionType',axis=1)

put_df = put_df.sort_values(['expiry_month','expiry_date'],ascending=False)

# put_df = put_df.set_index('instrument_name')
# call_df = call_df.set_index('instrument_name')

# put_df.set_index('instrument_name', inplace=True)
# call_df.set_index('instrument_name', inplace=True)

print(put_df.head())
a = put_df['expiry_month'].unique()



          instrument_name  mark_price  volume  estimated_delivery_price  \
130  BTC-30SEP22-100000-P    1.328187     0.0                  42161.61   
226  BTC-30SEP22-150000-P    2.446602     0.0                  42161.61   
228  BTC-30SEP22-200000-P    3.587070     0.0                  42161.61   
128   BTC-30SEP22-25000-P    0.042219     0.0                  42161.61   
102  BTC-30SEP22-250000-P    4.731448     0.0                  42161.61   

    expiry_date expiry_month  
130     30/9/22            9  
226     30/9/22            9  
228     30/9/22            9  
128     30/9/22            9  
102     30/9/22            9  
['9' '6' '4' '3' '2' '12']


In [162]:

dataframes = []
for region, df_region in put_df.groupby('expiry_date'):
    dataframes.append([region,df_region])

In [175]:
len(dataframes[6][1])


7

## Eporting to Excel

In [ ]:
# file_name = 'prueba.xlsx'

# put_df.to_excel('C:/Users/Alpha/My Drive/codingProjects/deribit/' + file_name)

In [ ]:
from openpyxl import load_workbook
 
file_name = 'nada.xlsx'
 
#load excel file
workbook = load_workbook(filename= 'C:/Users/Alpha/My Drive/codingProjects/deribit/' + file_name)
 
#open workbook
sheet = workbook.active
rangeExcel = 1
#modify the desired cell
for j in range(1,len(dataframes)):
    
    sheet["A" + str(range)] = dataframes[j][1].iloc[j]
    sheet["D4"] = '=D17+$C$2'
    rangeExcel = len(dataframes[j][1])
#save the file
workbook.save(filename='C:/Users/Alpha/My Drive/codingProjects/deribit/' + file_name)